# Activity 8.02

Student: DC427182

In [51]:
# 02: importing the required libraries

import pandas as pd
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [52]:
# 03: importing bokeh libraries
from bokeh.plotting import figure, show
from ipywidgets import interact, widgets
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import viridis

In [53]:
# 04: Load our olympia2016_athletes.csv dataset from the Datasets folder and set up the interaction elements. 
# Scroll down until you reach the cell that says getting the max number of medals and athletes of all countries. 
# Extract the two numbers from the dataset.
dataset = pd.read_csv('C:/Users/User/Documents/GitHub/gradest-1/DATAVIS/tutorials/Tutorial08/Datasets/olympia2016_athletes.csv')
dataset.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [54]:
# 04: checking maximum number of medals and athletes

# Scroll down until you reach the cell that says getting the max number of medals and athletes of all countries. 
# Extract the two numbers from the dataset.

dataset['total_medals'] = dataset['gold'] + dataset['silver'] + dataset['bronze']
per_country = dataset.groupby('nationality').agg({'total_medals': 'sum', 'name': 'count'}).reset_index()
max_medals = per_country['total_medals'].max()
max_athletes = per_country['name'].max()
print('Maximal number of medals:', max_medals)
print('Maximal number of athletes:', max_athletes)

Maximal number of medals: 264
Maximal number of athletes: 567


In [55]:
# 05: Create widgets for IntSlider for the maximum number of athletes (orientation vertical) and IntSlider for the maximum number of medals (orientation horizontal).

max_athletes_slider = widgets.IntSlider(
    value=max_athletes,
    min=1,
    max=max_athletes,
    step=1,
    description='Max athletes:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='d'
)

max_medals_slider = widgets.IntSlider(
    value=max_medals,
    min=1,
    max=max_medals,
    step=1,
    description='Max medals:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


In [57]:
# 06: Set up the @interact method, which will display the complete visualization.
# The only code we will write here is to show the return value of the get_plot method that gets all the interaction element values as parameters.

@interact(max_athletes=max_athletes_slider, max_medals=max_medals_slider)
def get_plot(max_athletes, max_medals):
    # 08: First, filter our countries dataset that contains all the countries that placed athletes in the Olympic games. 
    # Check whether they have a lower or equal number of medals and athletes than our max values passed as arguments.
    filtered_data = per_country[(per_country['total_medals'] <= max_medals) & (per_country['name'] <= max_athletes)]
    
    # 09: Create our DataSource and use it for the tooltips and the printing of the circle glyphs.
    source = ColumnDataSource(data=dict(
        x=filtered_data['total_medals'],
        y=filtered_data['name'],
        country=filtered_data['nationality']
    ))
    
    # 10: After that, create a new plot using the figure method that has the following attributes: 
    # title set to Rio Olympics 2016 - Medal comparison, x_axis_label set to Number of Medals, and y_axis_label set to Num of Athletes.
    p = figure(title="Rio Olympics 2016 - Medal comparison",
               x_axis_label='Number of Medals',
               y_axis_label='Num of Athletes',
               tools="hover", tooltips="@country: @x medals, @y athletes")
    
    # Set up a palette so each country can be a different color
    countries = filtered_data['nationality'].unique()
    pallette = viridis(len(countries))
    p.scatter('x', 'y', size=10, source=source, color=factor_cmap('country', palette=pallette, factors=countries))
    
    # 12: When executing the cell that contains the @interact decorator, you will see a scatter plot that displays a circle for every country 
    # displaying additional information, such as the shortcode of the country, the number of athletes, and the number of gold, silver, and bronze medals.
    show(p)


interactive(children=(IntSlider(value=567, continuous_update=False, description='Max athletes:', max=567, min=…